## 1. Imports

In [1]:
from torch.utils.data import TensorDataset, DataLoader
import torch
import os

from utils.MyTransformer_lP import GPTLightning, GPT

from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import WandbLogger
import pytorch_lightning as pl

## 2. Dataset Definiton

In [2]:
data = torch.load('./data/dataset.ds')  # same as your code

X_train = data['X_train']  # don't .to(device) — Lightning will move automatically
Y_train = data['Y_train']
X_valid = data['X_valid']
Y_valid = data['Y_valid']
X_test  = data['X_test']
Y_test  = data['Y_test']

train_ds = TensorDataset(X_train, Y_train)
valid_ds = TensorDataset(X_valid, Y_valid)
test_ds  = TensorDataset(X_test, Y_test)

train_loader = DataLoader(train_ds, batch_size=2048, shuffle=True, num_workers=4, pin_memory=True)
val_loader   = DataLoader(valid_ds, batch_size=2048, shuffle=False, num_workers=4, pin_memory=True)
test_loader  = DataLoader(test_ds, batch_size=2048, shuffle=False, num_workers=4, pin_memory=True)

## 3. Model Definiton

In [3]:
experiment_name="GPT_pico" # Changes wandb expeirment name
project_name = "Spike-Synth-Surrogate" # Changes where wandb project
logging_directory = ".temp"
checkpoint_path = "models/SRNN" # where the final checkpoint is saved

lr = 0.005
max_epochs = 100
model_type = 'pico-test' 

In [4]:
# Build model config
model_config = GPT.get_default_config()
model_config.model_type =  model_type
model_config.block_size = X_train.shape[1]
model = GPTLightning(model_config, lr=lr, max_epochs=max_epochs)

number of parameters: 17k


In [5]:
script_dir = os.getcwd() 
logging_directory = os.path.join(script_dir, logging_directory)
logging_directory = os.path.abspath(logging_directory)
os.makedirs(logging_directory, exist_ok=True)
os.environ["WANDB_DIR"] = logging_directory

In [6]:
# Create a CometLogger instance
wandb_logger = WandbLogger(
                          log_model=True,
                          project=project_name,
                          name=experiment_name,
                          save_dir=logging_directory
                          )

# log gradients and model topology
wandb_logger.watch(model)
wandb_logger.experiment.log_code(".", include_fn=lambda path: path.endswith(".py") or path.endswith(".ipynb"))

wandb: Currently logged in as: lupos to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


<Artifact source-Spike-Synth-Surrogate-surrogate_2_create_gpt_surrogate.ipynb>

## 4. Training

In [ ]:
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
     filename=experiment_name + str("gpt_model-{epoch:02d}-{val_loss:.2f}"),
    save_last=True,
)

early_stop_callback = EarlyStopping(monitor='val_loss', patience=10, mode='min', verbose=True)

trainer = pl.Trainer(
    max_epochs=max_epochs,
    accelerator="gpu" if torch.cuda.is_available() else "cpu", 
    callbacks=[checkpoint_callback, early_stop_callback],
    logger=wandb_logger,
    log_every_n_steps=10,
)

# Results in crashes
# model = torch.compile(model)

trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

wandb_logger.finalize("sucess")

# Testing
# trainer.test(model, dataloaders=test_loader)

# After training the best checkpoint path is:
print("Best checkpoint saved at:", checkpoint_callback.best_model_path)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name    | Type    | Params | Mode 
--------------------------------------------
0 | model   | GPT     | 17.1 K | train
1 | loss_fn | MSELoss | 0      | train
--------------------------------------------
17.1 K    Trainable params
0         Non-trainable params
17.1 K    Total params
0.068     Total estimated model params size (MB)
35        Modules in train mode
0         Modules in eval mode


Sanity Checking: |     | 0/? [00:00<?, ?it/s]

/nix/store/mxb5f60mz822vg50ll0pz7063spw4bnr-python3-3.12.11-env/lib/python3.12/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/nix/store/mxb5f60mz822vg50ll0pz7063spw4bnr-python3-3.12.11-env/lib/python3.12/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Training: |            | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Metric val_loss improved by 0.001 >= min_delta = 0.0. New best score: 0.053


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]